# О задании

## Тема
Объединение таблиц и сложные выборки

## Цель домашнего задания
- научиться работать с интервальными таблицы, уметь их джойнить с другими таблицами;
- научиться обрабатывать транзакции относительно какого-то промежутка времени.
 

## Условия
Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.

**За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**

Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

<font size=4>[**Шаг 1**](#Шаг-1)</font>

(Балл - 2) Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу *transaction_bd* со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции

(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции

(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента

(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала

(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу *customer_bd* со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента

(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте

(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте

(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента

(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

**Примечание для следующих шагов:** сумма транзакций не может быть null-ом, это всегда число.

<font size=4>[**Шаг 2**](#Шаг-2)</font>

**(Балл - 1 за каждый пункт)** Написать скрипты:

- a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
- d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
- e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

<font size=4>[**Шаг 3**](#Шаг-3)</font>

**(Балл - 3)** Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
 

## Сроки выполнения задания
До 09.04.2023, 23:59.

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
import re

Создадим функции для подключения к базе данных, выполнения запросов и создании таблиц.

In [4]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [4]:
def sql_insert(con, entities, text_sql_insert_into_table):

    cursorObj = con.cursor()
    cursorObj.execute(text_sql_insert_into_table, entities)

## Шаг 1

Подключимся к базе данных

In [5]:
connection = create_connection('database.db')

Connection to SQLite DB successful


Создадим таблицу transaction_bd

In [32]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS transaction_bd (
                                            "TRANSACTION_ID" INTEGER,
                                            "TX_DATETIME" NUMERIC,
                                            "CUSTOMER_ID" INTEGER,
                                            "TERMINAL_ID" INTEGER,
                                            "TX_AMOUNT" REAL
                                                        )
                     '''

In [33]:
execute_query(connection, sql_create_table)

Query executed successfully


Создадим таблицу customer_bd

In [34]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS customer_bd (
                                            "CLIENT_ID" INTEGER,
                                            "START_DT" NUMERIC,
                                            "END_DT" NUMERIC,
                                            "CLIENT_NAME" TEXT,
                                            "YEAR_BIRTH" TEXT
                                                        )
                     '''

In [35]:
execute_query(connection, sql_create_table)

Query executed successfully


Прочитаем данные из файлов и вставим их в таблицы

In [36]:
text_sql_insert_into_table = '''
                                 INSERT INTO transaction_bd(
                                                         TRANSACTION_ID,
                                                         TX_DATETIME,
                                                         CUSTOMER_ID,
                                                         TERMINAL_ID,
                                                         TX_AMOUNT
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [37]:
with open('transactions_for_dz2.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(','), text_sql_insert_into_table)
connection.commit()

In [38]:
text_sql_insert_into_table = '''
                                 INSERT INTO customer_bd(
                                                         START_DT,
                                                         END_DT,
                                                         CLIENT_NAME,
                                                         YEAR_BIRTH,
                                                         CLIENT_ID
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [39]:
with open('client_info.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(';'), text_sql_insert_into_table)
connection.commit()

## Шаг 2
Напишем скрипты:

a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [8]:
actual_date = '2023-05-01'

In [9]:
sql_task_a = f'''
            WITH
            
            actual_cusomers as
                (SELECT
                    CLIENT_ID
                FROM
                    customer_bd
                WHERE END_DT >= {actual_date}),
                
            sum_total as
                (SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE
                    CUSTOMER_ID in actual_cusomers
                GROUP by CUSTOMER_ID),

                max_total as
                (SELECT
                    *,
                    max(TOTAL_AMOUNT) over () as MAX_TOTAL_AMOUNT

                FROM
                    sum_total
                )


            SELECT
                CLIENT_NAME
            FROM
                max_total as m
            JOIN customer_bd as c on c.CLIENT_ID = m.CUSTOMER_ID
            WHERE TOTAL_AMOUNT = MAX_TOTAL_AMOUNT


            '''
data = pd.read_sql(sql_task_a, connection)
data

,CLIENT_NAME
0,Kylee


b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [21]:
sql_task_b = f'''
            WITH 
                      
            actual_cusomers as
                (SELECT
                    CLIENT_ID
                FROM
                    customer_bd
                WHERE END_DT >= {actual_date}),
                
            sum_total as
                (SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE
                    CUSTOMER_ID in actual_cusomers
                GROUP by CUSTOMER_ID),

                min_total as
                (SELECT
                    *,
                    min(TOTAL_AMOUNT) over () as MIN_TOTAL_AMOUNT

                FROM
                    sum_total
                )


            SELECT
                CLIENT_NAME
            FROM
                min_total as m
            JOIN customer_bd as c on c.CLIENT_ID = m.CUSTOMER_ID
            WHERE TOTAL_AMOUNT = MIN_TOTAL_AMOUNT


            '''
data = pd.read_sql(sql_task_b, connection)
data

,CLIENT_NAME
0,Kloe


c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.

In [14]:
actual_date = '2023-04-01'

In [23]:
sql_task_c = f'''
                WITH 

                actual_cusomer as
                    (SELECT
                        CLIENT_ID
                    FROM
                        customer_bd
                    WHERE END_DT >= '2023-04-01')

                SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE TX_DATETIME < '2023-04-01' AND CUSTOMER_ID in actual_cusomer AND substr(CUSTOMER_ID, 1, 3) = "111"
                GROUP by CUSTOMER_ID
                '''
data = pd.read_sql(sql_task_c, connection)
data

,CUSTOMER_ID,TOTAL_AMOUNT
0,1112,35273.16
1,1113,229558.85
2,1114,108622.52
3,1115,38690.76
4,1116,191203.59
5,1117,52751.58
6,1118,353003.39
7,1119,221192.71


In [17]:
sql_task_c

'\n                WITH \n\n                    actual_cusomers as\n                        (SELECT\n                            CLIENT_ID\n                        FROM\n                            customer_bd\n                        WHERE END_DT >= 2023-04-01)\n\n                SELECT\n                    CUSTOMER_ID,\n                    sum(TX_AMOUNT) as TOTAL_AMOUNT\n                FROM\n                    transaction_bd\n                where TX_DATETIME < 2023-04-01 AND CUSTOMER_ID in actual_cusomers AND substr(CUSTOMER_ID, 1, 3) = "111"\n                GROUP by CUSTOMER_ID\n                '

d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [6]:
connection.close()

## Шаг 3

In [135]:
connection.close()